Данный файл содержит предобработку данных.
Датасет POSEIDON.csv содержал запятые в числовых значениях и группу соединений в колонке с "Грузом", перечисленных через запятую, из-за чего датасет не отображался правильно. Лишние запятые были удалены в Excel.

Этот файл содержит два итоговых файла.
- poseidon_Fluorescence.csv, который далее используется для кластеризации и эмбеддингов.
- Hakaton_data.csv, для которого рассчитываются физико-химические дескрипторы


In [ ]:
#Импорт библиотек
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pylab as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Подключение к гугл-коллабу, иногда ячейка отсутствует
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Метод позволяет прочитать преподготовленный .csv файл
import chardet

with open('/content/drive/MyDrive/Datacon/POSEIDON_4.csv', 'rb') as f:
    result=chardet.detect(f.read())

df = pd.read_csv('/content/drive/MyDrive/Datacon/POSEIDON_4.csv', index_col=0, encoding=result['encoding'], sep='\;')

<ipython-input-7-55454854a29b>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('/content/drive/MyDrive/Datacon/POSEIDON_4.csv', index_col=0, encoding=result['encoding'], sep='\;')


In [ ]:
#Смотрим на данные
df

,Cell line,Cargo,PubmedID,Uptake,Units,Conc.,Time,Temp.,Method,Type,Sequence,time
Peptide,,,,,,,,,,,,
Tat (49-57),Jurkat cells,Fluorescein,11087855,650.00,Mean Fluorescence intensity,12.5 uM,15 min,23ÂºC,Flow cytometry,Cellular uptake,RKKRRQRRR,#ÇÍÀ×!
Tat (49-56),Jurkat cells,Fluorescein,11087855,31.25,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,RKKRRQRR,NaN
Tat (49-55),Jurkat cells,Fluorescein,11087855,10.00,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,RKKRRQR,NaN
Tat (50-57),Jurkat cells,Fluorescein,11087855,22.50,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,KKRRQRRR,NaN
Tat (51-57),Jurkat cells,Fluorescein,11087855,20.00,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,KRRQRRR,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Oleyl-R2-(HR)4,MDA-MB-231 cells,siRNA-A490,35456715,100,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,NaN
Oleyl-R3-(HR)4,MDA-MB-231 cells,siRNA-A491,35456715,250,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,NaN
Oleyl-R4-(HR)4,MDA-MB-231 cells,siRNA-A492,35456715,400,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,NaN


In [ ]:
#перезаписываем данные для удобства, чтобы "Peptide" стала колонкой
df.to_csv("/content/drive/MyDrive/Datacon/data_first.csv", index = True)
df=pd.read_csv("/content/drive/MyDrive/Datacon/data_first.csv")
df

In [ ]:
df

,Peptide,Cell line,Cargo,PubmedID,Uptake,Units,Conc.,Time,Temp.,Method,Type,Sequence,time
0,Tat (49-57),Jurkat cells,Fluorescein,11087855,650.00,Mean Fluorescence intensity,12.5 uM,15 min,23ÂºC,Flow cytometry,Cellular uptake,RKKRRQRRR,#ÇÍÀ×!
1,Tat (49-56),Jurkat cells,Fluorescein,11087855,31.25,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,RKKRRQRR,NaN
2,Tat (49-55),Jurkat cells,Fluorescein,11087855,10.00,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,RKKRRQR,NaN
3,Tat (50-57),Jurkat cells,Fluorescein,11087855,22.50,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,KKRRQRRR,NaN
4,Tat (51-57),Jurkat cells,Fluorescein,11087855,20.00,Mean Fluorescence intensity,12.5 uM,10 min,23ÂºC,Flow cytometry,Cellular uptake,KRRQRRR,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,Oleyl-R2-(HR)4,MDA-MB-231 cells,siRNA-A490,35456715,100,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,NaN
2367,Oleyl-R3-(HR)4,MDA-MB-231 cells,siRNA-A491,35456715,250,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,NaN
2368,Oleyl-R4-(HR)4,MDA-MB-231 cells,siRNA-A492,35456715,400,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,NaN
2369,Oleyl-R5-(HR)4,MDA-MB-231 cells,siRNA-A493,35456715,500,Mean Fluorescence intensity,50 nM,24h,37ÂºC,Flow cytometry,Cellular internalization of oleyl,RRRRRHRHRHRHR,NaN


In [ ]:
#Унифицируем значения температуры, для пропущенных значений предполагаем, что исследователи считают нормальной температурой, которую нет нужды указывать t=36ºC
def temp_split(temp_str):
    try:
        temp=str(temp_str).split()
        temp_=temp[0]
        if 'ÂºC' in temp_:
            te=float(temp_.replace('ÂºC', ''))
        return te
    except:
        te=float(36)
        return
df['Temp.']=df['Temp.'].apply(temp_split)
df

,Peptide,Cell line,Cargo,PubmedID,Uptake,Units,Conc.,Time,Temp.,Method,Type,Sequence,time
0,Tat (49-57),Jurkat cells,Fluorescein,11087855,650.00,Mean Fluorescence intensity,12.5 uM,15 min,23.0,Flow cytometry,Cellular uptake,RKKRRQRRR,#ÇÍÀ×!
1,Tat (49-56),Jurkat cells,Fluorescein,11087855,31.25,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,RKKRRQRR,NaN
2,Tat (49-55),Jurkat cells,Fluorescein,11087855,10.00,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,RKKRRQR,NaN
3,Tat (50-57),Jurkat cells,Fluorescein,11087855,22.50,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,KKRRQRRR,NaN
4,Tat (51-57),Jurkat cells,Fluorescein,11087855,20.00,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,KRRQRRR,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,Oleyl-R2-(HR)4,MDA-MB-231 cells,siRNA-A490,35456715,100,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,NaN
2367,Oleyl-R3-(HR)4,MDA-MB-231 cells,siRNA-A491,35456715,250,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,NaN
2368,Oleyl-R4-(HR)4,MDA-MB-231 cells,siRNA-A492,35456715,400,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,NaN
2369,Oleyl-R5-(HR)4,MDA-MB-231 cells,siRNA-A493,35456715,500,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRRHRHRHRHR,NaN


In [ ]:
#Переводим время из минут в часы
def split_hour(hour_str):
    try:
        hour=str(hour_str).split()
        hour_=hour[0]
        if 'min' in hour:
            ho=float(hour_.replace('min', ''))/60
        elif ', min' in hour:
            ho=float(hour_.replace(', min', ''))/60
        elif 'h' in hour_:
            ho=float(hour_.replace('h', ''))
        return ho
    except:
        return None
df['Time, h']=df['Time'].apply(split_hour)
df

,Peptide,Cell line,Cargo,PubmedID,Uptake,Units,Conc.,Time,Temp.,Method,Type,Sequence,time,"Time, h"
0,Tat (49-57),Jurkat cells,Fluorescein,11087855,650.00,Mean Fluorescence intensity,12.5 uM,15 min,23.0,Flow cytometry,Cellular uptake,RKKRRQRRR,#ÇÍÀ×!,0.250000
1,Tat (49-56),Jurkat cells,Fluorescein,11087855,31.25,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,RKKRRQRR,NaN,0.166667
2,Tat (49-55),Jurkat cells,Fluorescein,11087855,10.00,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,RKKRRQR,NaN,0.166667
3,Tat (50-57),Jurkat cells,Fluorescein,11087855,22.50,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,KKRRQRRR,NaN,0.166667
4,Tat (51-57),Jurkat cells,Fluorescein,11087855,20.00,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,KRRQRRR,NaN,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,Oleyl-R2-(HR)4,MDA-MB-231 cells,siRNA-A490,35456715,100,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,NaN,24.000000
2367,Oleyl-R3-(HR)4,MDA-MB-231 cells,siRNA-A491,35456715,250,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,NaN,24.000000
2368,Oleyl-R4-(HR)4,MDA-MB-231 cells,siRNA-A492,35456715,400,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,NaN,24.000000
2369,Oleyl-R5-(HR)4,MDA-MB-231 cells,siRNA-A493,35456715,500,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRRHRHRHRHR,NaN,24.000000


In [ ]:
#Переводим единицы измерения концентрации в uM и удаляем единицы измерения из ячеек
def split_conc(conc_str):
    try:
        conc=str(conc_str).split()
        conc_=conc[0]
        if ' uM' in conc:
            c=float(conc_.replace(' uM', ''))
        elif 'uM' in conc:
            c=float(conc_.replace('uM', ''))
        elif 'umol/L' in conc:
            c=float(conc_.replace('umol/L', ''))
        elif ' umol/L' in conc:
            c=float(conc_.replace(' umol/L', ''))
        elif 'nM' in conc:
            c=float(conc_.replace('nM', ''))*1000
        elif '_nM' in conc:
            c=float(conc_.replace('_nM', ''))*1000
        return c
    except:
        return None
df['Conc., uM']=df['Conc.'].apply(split_conc)
df

,Peptide,Cell line,Cargo,PubmedID,Uptake,Units,Conc.,Time,Temp.,Method,Type,Sequence,time,"Time, h","Conc., uM"
0,Tat (49-57),Jurkat cells,Fluorescein,11087855,650.00,Mean Fluorescence intensity,12.5 uM,15 min,23.0,Flow cytometry,Cellular uptake,RKKRRQRRR,#ÇÍÀ×!,0.250000,12.5
1,Tat (49-56),Jurkat cells,Fluorescein,11087855,31.25,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,RKKRRQRR,NaN,0.166667,12.5
2,Tat (49-55),Jurkat cells,Fluorescein,11087855,10.00,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,RKKRRQR,NaN,0.166667,12.5
3,Tat (50-57),Jurkat cells,Fluorescein,11087855,22.50,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,KKRRQRRR,NaN,0.166667,12.5
4,Tat (51-57),Jurkat cells,Fluorescein,11087855,20.00,Mean Fluorescence intensity,12.5 uM,10 min,23.0,Flow cytometry,Cellular uptake,KRRQRRR,NaN,0.166667,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2366,Oleyl-R2-(HR)4,MDA-MB-231 cells,siRNA-A490,35456715,100,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,NaN,24.000000,50000.0
2367,Oleyl-R3-(HR)4,MDA-MB-231 cells,siRNA-A491,35456715,250,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,NaN,24.000000,50000.0
2368,Oleyl-R4-(HR)4,MDA-MB-231 cells,siRNA-A492,35456715,400,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,NaN,24.000000,50000.0
2369,Oleyl-R5-(HR)4,MDA-MB-231 cells,siRNA-A493,35456715,500,Mean Fluorescence intensity,50 nM,24h,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRRHRHRHRHR,NaN,24.000000,50000.0


In [ ]:
'''Унифицируем значения таргетной величины, оставляем только флюоресценцию.
#Нами был проведен поиск формул перевода других единиц измерения во флюоресценцию,
но для каждого отдельного случая необходимо было проверять необходимые для перевода значения в статьях,
что заняло бы дня три.
Во время поиска переводов, мы нашли статью "POSEIDON: Peptidic Objects SEquence-based Interaction with cellular DOmaiNs: a new database and predictor"
Preto et al. Journal of Cheminformatics (2024) 16:18 https://doi.org/10.1186/s13321-024-00810-7
В статье была проведена работа по созданию регрессионной модели машинного обучения.
На их дескрипторах, рассчитанных для CPP с таргетной величиной флюоресценции были получены отличные результаты.
Мы решили строить свою модель только на значениях флюоресценции, но использовать другие дескрипторы.
В статье в качестве дескрипторов использовались двоичные кодировки информации о клеточной линии,
доставляемом грузе, аминокислотной последовательности пептида, температуре, времени доставки и концентрации пептида.
Мы в нашей работе, в отличие от ранее опубликованной работы, собираемся проверить:
- физико-химические дескрипторы
- эмбеддинги (BERT, MISTRAL)
'''
def split_uptake(uptak, units_):
    """

    Args:
      uptak:
      units_:

    Returns:

    """
    try:
        unit=str(units_)

        if 'Mean Fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Fluorescence intensity (%)' in unit:
            up=float(uptak)/100
        elif 'Normalized Mean Fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Intracellular Fluorescence' in unit:
            up=float(uptak)
        elif 'Fluorescence (fold/basal)' in unit:
            up=float(uptak)
        elif 'Geo Mean Fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Mean Fluorescence intensity/mg protein' in unit:
            up=float(uptak)
        elif 'Mean Fluorescence intensity/cell' in unit:
            up=float(uptak)
        elif 'Median Normalized Fluorescence' in unit:
            up=float(uptak)
        elif 'Relative fluorescence' in unit:
            up=float(uptak)
        elif 'Relative Mean Fluorescence intensity (%)' in unit:
            up=float(uptak)/100
        elif 'Average Median Fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Hoechst fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Microscopic fluorescence intensity' in unit:
            up=float(uptak)
        elif 'Relative Median Fluorescence' in unit:
            up=float(uptak)
        elif 'Relative Mean Fluorescence intensity (%)' in unit:
            up=float(uptak)/100
        return up
    except:
        return None

for i in range(len(df['Uptake'])):
    df['target']=split_uptake(df['Uptake'][i], df['Units'][i])
for i in range(len(df['Uptake'])):
    df['target'][i]=split_uptake(df['Uptake'][i], df['Units'][i])
df['target']

'''     elif 'Relative penetration efficiency' in unit:
            up=float(uptak)
        elif '% Intracellular fluorescence of CyLoP-1' in unit:
            up=float(uptak)
        elif 'OD (optical density values)' in unit:
            up=float(uptak)
        elif '% Absorbance relative to peptide 1 (405 nm)' in unit:
            up=float(uptak)/100
        elif 'Cellular uptake (%)' in unit:
            up=float(uptak)/100
        elif 'Cellular uptake' in unit:
            up=float(uptak)
        elif 'Cellular retention (mol/cell x 10-16)' in unit:
            up=float(uptake)
        elif 'pmol internalized peptide / mg protein' in unit:
            up=float(uptake)
        elif 'pmol/mg protein Cellular uptake' in unit:
            up=float(uptake)
        elif 'Mnlog FITC' in unit:
            up=float(uptake)
        elif 'nmol/mg protein' in unit:
            up=float(uptake)
        elif 'Fold Change in GeoMean Fluorescence of control sample' in unit:
            up=float(uptake)
        elif 'Relative Cellular uptake' in unit:
            up=float(uptake)
        elif '% Amount transported into basolateral fluid' in unit:
            up=float(uptake)
        elif 'RFU/mg' in unit:
            up=float(uptake)
        elif 'pmol/ug total protein' in unit:
            up=float(uptake)
        elif 'Internalized peptide (uM)' in unit:
            up=float(uptake)
        elif 'pmol/uM Internalized peptide' in unit:
            up=float(uptake)
        elif '% Positive cells' in unit:
            up=float(uptake)
        elif 'Median TAMRA intensity' in unit:
            up=float(uptake)
        elif 'Internalization' in unit:
            up=float(uptake)
        elif 'pg PO/ 1000 cells' in unit:
            up=float(uptake)
'''

Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-18-d3582b964ac8>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][i]=split_uptake(df['Uptake'][i], df['Units'][i])
<ipython-input-18-d3582b964ac8>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'][i]=split_uptake(df['Uptake'][i], df['Units'][i])
<ipython-input-18-d3582b964ac8>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d

"     elif 'Relative penetration efficiency' in unit:\n            up=float(uptak)\n        elif '% Intracellular fluorescence of CyLoP-1' in unit:\n            up=float(uptak)\n        elif 'OD (optical density values)' in unit:\n            up=float(uptak)\n        elif '% Absorbance relative to peptide 1 (405 nm)' in unit:\n            up=float(uptak)/100\n        elif 'Cellular uptake (%)' in unit:\n            up=float(uptak)/100\n        elif 'Cellular uptake' in unit:\n            up=float(uptak)\n        elif 'Cellular retention (mol/cell x 10-16)' in unit:\n            up=float(uptake)\n        elif 'pmol internalized peptide / mg protein' in unit:\n            up=float(uptake)\n        elif 'pmol/mg protein Cellular uptake' in unit:\n            up=float(uptake)\n        elif 'Mnlog FITC' in unit:\n            up=float(uptake)\n        elif 'nmol/mg protein' in unit:\n            up=float(uptake)\n        elif 'Fold Change in GeoMean Fluorescence of control sample' in unit:\n

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2371 entries, 0 to 2370
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Peptide    2370 non-null   object 
 1   Cell line  2357 non-null   object 
 2   Cargo      2305 non-null   object 
 3   PubmedID   2371 non-null   int64  
 4   Uptake     2371 non-null   object 
 5   Units      2333 non-null   object 
 6   Conc.      2143 non-null   object 
 7   Time       2276 non-null   object 
 8   Temp.      1661 non-null   float64
 9   Method     2362 non-null   object 
 10  Type       2360 non-null   object 
 11  Sequence   2367 non-null   object 
 12  time       1 non-null      object 
 13  Time, h    2063 non-null   float64
 14  Conc., uM  1785 non-null   float64
 15  target     1288 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory usage: 296.5+ KB


In [ ]:
#Удаляем столбцы, которые не будем никак учитывать при обучении
#Мы посчитали, что груз не должен быть важен для нашей модели, так как большее значение имеет вторичная и третичная структура CPP,
#а не малые флуктуации структуры, связанные с различным переносимым грузом, поэтому мы удалили столбец Cargo.
#Клеточные линии также были удалены, так как несмотря на различия в строении клеточной стенки для разных культур клеток, эти различия достаточно незначительны,
#а больший вклад несет именно структура CPP
df_new=df.drop(columns=['Peptide', 'Cell line',	'Cargo', 'PubmedID', 'Conc.', 'Time', 'time'])
df_new

,Uptake,Units,Temp.,Method,Type,Sequence,"Time, h","Conc., uM",target
0,650.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQRRR,0.250000,12.5,650.0
1,31.25,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQRR,0.166667,12.5,31.25
2,10.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQR,0.166667,12.5,10.0
3,22.50,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,KKRRQRRR,0.166667,12.5,22.5
4,20.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,KRRQRRR,0.166667,12.5,20.0
...,...,...,...,...,...,...,...,...,...
2366,100,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,24.000000,50000.0,100.0
2367,250,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,24.000000,50000.0,250.0
2368,400,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,24.000000,50000.0,400.0
2369,500,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRRHRHRHRHR,24.000000,50000.0,500.0


In [ ]:
#Здесь записывается файл poseidon_Fluorescence.csv и отправляется на дообработку и расчет эмбеддингов
#Маневр для распараллеливания работы
df_new.to_csv("/content/drive/MyDrive/Datacon/poseidon_Fluorescence.csvv", index = True)

In [ ]:
#Удаляю строки с пустыми значениями в строках с таргетной величиной и последовательности АК CPP
df_new = df_new.dropna(subset=['Sequence'])
df_new = df_new.dropna(subset=['target'])
df_new.reset_index(drop= True , inplace= True )

In [ ]:
df_new

,Uptake,Units,Temp.,Method,Type,Sequence,"Time, h","Conc., uM",target
0,650.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQRRR,0.250000,12.5,650.0
1,31.25,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQRR,0.166667,12.5,31.25
2,10.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQR,0.166667,12.5,10.0
3,22.50,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,KKRRQRRR,0.166667,12.5,22.5
4,20.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,KRRQRRR,0.166667,12.5,20.0
...,...,...,...,...,...,...,...,...,...
1279,50,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RHRHRHRHR,24.000000,50000.0,50.0
1280,100,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,24.000000,50000.0,100.0
1281,250,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,24.000000,50000.0,250.0
1282,400,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,24.000000,50000.0,400.0


In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284 entries, 0 to 1283
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Uptake     1284 non-null   object 
 1   Units      1284 non-null   object 
 2   Temp.      946 non-null    float64
 3   Method     1283 non-null   object 
 4   Type       1283 non-null   object 
 5   Sequence   1284 non-null   object 
 6   Time, h    1120 non-null   float64
 7   Conc., uM  961 non-null    float64
 8   target     1284 non-null   object 
dtypes: float64(3), object(6)
memory usage: 90.4+ KB


In [ ]:
#Заполняю пустые значения методом knn
import numpy as np
from sklearn.impute import KNNImputer
knn=df_new.drop(columns=['Uptake',	'Units',	'Method',	'Type',	'Sequence', 'target'])
imputer = KNNImputer(n_neighbors = 5, weights = 'uniform')
knn_imputer=pd.DataFrame(imputer.fit_transform(knn))

In [ ]:
#Записываю полученные значения в датафрейм
df_new['Temp.']=knn_imputer[0]
df_new['Time, h']=knn_imputer[1]
df_new['Conc., uM']=knn_imputer[2]

In [ ]:
df_new

,Uptake,Units,Temp.,Method,Type,Sequence,"Time, h","Conc., uM",target
0,650.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQRRR,0.250000,12.5,650.0
1,31.25,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQRR,0.166667,12.5,31.25
2,10.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,RKKRRQR,0.166667,12.5,10.0
3,22.50,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,KKRRQRRR,0.166667,12.5,22.5
4,20.00,Mean Fluorescence intensity,23.0,Flow cytometry,Cellular uptake,KRRQRRR,0.166667,12.5,20.0
...,...,...,...,...,...,...,...,...,...
1279,50,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RHRHRHRHR,24.000000,50000.0,50.0
1280,100,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRHRHRHRHR,24.000000,50000.0,100.0
1281,250,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRHRHRHRHR,24.000000,50000.0,250.0
1282,400,Mean Fluorescence intensity,37.0,Flow cytometry,Cellular internalization of oleyl,RRRRHRHRHRHR,24.000000,50000.0,400.0


In [ ]:
#Записываю файл Hakaton_data.csv и отправляю его на расчет дескрипторов
df_new.to_csv("/content/drive/MyDrive/Datacon/Hakaton_data.csv", index = True)